# TD et TME 9: CREATION DE SCHEMAS-MODIFICATION DES DONNEES


## Installation H2 

Le système utilisé pendant les TME est H2.

In [2]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Télécharger le pilote de H2

In [3]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

--2024-04-14 22:17:39--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081604 (1,0M) [application/octet-stream]
Saving to: ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1,03M  --.-KB/s    in 0,01s   

2024-04-14 22:17:39 (96,3 MB/s) - ‘postgresql-42.6.0.jar’ saved [1081604/1081604]

--2024-04-14 22:17:39--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2543011 (2,4M) [application/octet-stream]
Saving to: ‘h2-2.1.214.jar’

h2-2.1.214.jar      100%[===================>]   2,42M  --.-KB/s    in 0,02s   

2024-04-14 22:17:

**Relancez le kernel**: Kernel -> Restart kernel ...

https://www.psycopg.org/docs/
http://localhost:8082

In [4]:
import psycopg2
import os
local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

### Démarrer le serveur SQL H2

In [5]:
#Remplacez les valeurs des variables suivantes
#NN= numéro de groupe: chaine de caractères (ex: 01, 02,.., 13), remplacer xx 
NN = '06'
#BB=numero binome: chaîne de caractères (assigné par l'enseignant, ex: 01,..., 15), remplacer xx
BB = '13'
port = 5000+int(NN+BB)
print(f'Le numero du port utilisé pour la connexion à la BD est: {port}')

Le numero du port utilisé pour la connexion à la BD est: 5613


In [6]:
%%bash --bg --out output -s "$port"
java -Dh2.bindAddress=127.0.0.1 -cp h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort $1 -baseDir ./data -ifNotExists

## Fonctions utiles

### connect_H2

In [7]:
def connect_H2(db,user,password):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

### execute

In [8]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount and cursor.description is not None:
            names = [desc[0] for desc in cursor.description]
            lengths={}
            for attr in names:
                lengths[attr]=len(attr)
            for ligne in cursor:
                i=0
                for attr in ligne:
                    lengths[names[i]]=max(lengths[names[i]],len(str(attr).replace('\n',' ')))
                    i=i+1
            print('|',end='')
            for attr in names:
                print(str(attr).ljust(lengths[attr]),end='|')
            print()
            print('|',end='')
            for attr in names:
                print(''.ljust(lengths[attr]+1,'-'),end='')
            print()
            cursor.execute(query)
            for ligne in cursor:
                i=0
                print('|',end='')
                for attr in ligne:
                    print(str(attr).replace('\n',' ')[:lengths[names[i]]].ljust(lengths[names[i]]),end='|')
                    i=i+1
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

### show_table

In [9]:
def show_table(connection,table_name):

    query=f"""
SELECT table_name,
       column_name,
       data_type,
       column_default,
       is_nullable,
       character_maximum_length,
       numeric_precision
FROM information_schema.columns 
where lower(table_name)  = '{table_name}'  
"""
    execute(connection,query)
    

### show_schema

In [10]:
def show_schema(connection):

    print('*********** Tables ************')
    query="""
    select TABLE_NAME
    from INFORMATION_SCHEMA.TABLES 
    where TABLE_SCHEMA = 'public'
    """
    execute(connection,query,show=True)

    print('\n\n*********** Domaines ************')
    query="""
    SELECT domain_name,check_clause 
    FROM information_schema.domain_constraints a, information_schema.check_constraints b
    where a.constraint_name=b.constraint_name
    """
    execute(connection,query)

    print('\n\n*********** Attributs ************')
    query=f"""
    SELECT c.table_name,
           c.column_name,
           c.data_type,
           c.column_default,
           c.is_nullable,
           c.character_maximum_length,
           c.numeric_precision
    FROM INFORMATION_SCHEMA.TABLES t, information_schema.columns c
    where t.table_name=c.table_name
    and t.TABLE_SCHEMA = 'public'
    """
    execute(connection,query)
    

### Connexion à H2

In [11]:
connection = connect_H2("jo"+f'{port}',"ba","ba")

Connection to H2 DB successful


## Exercices TD

pour supprimer le schéma de la base courante

In [12]:
query="""
DROP ALL OBJECTS
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc8246b80; closed: -1>

### VILLES ET PAYS
On veut créer un schéma relationnel pour stocker des informations sur des villes et des pays.



#### Traduisez le schéma relationnel suivant en instructions SQL:


- Ville(<u>nom</u>, population, pays*) 
- Pays(<u>nom</u>, capitale*)

où pays est une référence vers un pays dans la table Pays et capitale est une référence vers une ville dans la table Ville.

In [13]:
query="""
create table Ville(
    nom varchar(50) primary key,
    population numeric(10),
    pays varchar(50)
);

create table Pays(
    nom varchar(50) primary key,
    capitale varchar(50),
    constraint fk_ville foreign key(capitale) references Ville(nom)
);

alter table ville add constraint fk_pays foreign key(pays) references Pays(nom)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc82468b0; closed: -1>

Interroger le catalogue de données

In [14]:
show_schema(connection)

*********** Tables ************
2 rows
|table_name|
|-----------
|ville     |
|pays      |


*********** Domaines ************
0 rows


*********** Attributs ************
5 rows
|table_name|column_name|data_type        |column_default|is_nullable|character_maximum_length|numeric_precision|
|---------------------------------------------------------------------------------------------------------------
|ville     |nom        |character varying|None          |NO         |50                      |None             |
|ville     |population |numeric          |None          |YES        |None                    |10               |
|ville     |pays       |character varying|None          |YES        |50                      |None             |
|pays      |nom        |character varying|None          |NO         |50                      |None             |
|pays      |capitale   |character varying|None          |YES        |50                      |None             |


#### Insérez la France avec sa capitale Paris (3 millions d'habitants) dans la base de données (deux insertions).

In [15]:
query="""
insert into Pays values('France', null);
insert into Ville values('Paris',3000000,'France');
update Pays set capitale = 'Paris' where nom = 'France'
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc82467c0; closed: -1>

In [16]:
query="""
select * from Ville
"""
execute(connection,query,show=True)

1 rows
|nom  |population|pays  |
|------------------------
|Paris|3000000   |France|


<cursor object at 0x7facc0480040; closed: -1>

In [17]:
query="""
select * from Pays
"""
execute(connection,query,show=True)

1 rows
|nom   |capitale|
|----------------
|France|Paris   |


<cursor object at 0x7facc0480130; closed: -1>

#### Modifiez le schéma de telle manière que la suppression d'un pays déclenche automatiquement la suppression de toutes les villes du pays.

In [18]:
query="""
alter table Ville drop constraint fk_pays;
alter table Ville add constraint fk_pays foreign key(pays) references Pays(nom) on delete cascade
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc0480400; closed: -1>

####  Effacez les Pays 

In [19]:
query="""
DELETE FROM Pays
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc0480220; closed: -1>

In [20]:
query="""
select * from Pays
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc04804f0; closed: -1>

#### Exécutez la requête suivante; pourquoi Paris a disparu ?

In [21]:
query="""
select * from Ville
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc04805e0; closed: -1>

In [22]:
show_schema(connection)

*********** Tables ************
2 rows
|table_name|
|-----------
|ville     |
|pays      |


*********** Domaines ************
0 rows


*********** Attributs ************
5 rows
|table_name|column_name|data_type        |column_default|is_nullable|character_maximum_length|numeric_precision|
|---------------------------------------------------------------------------------------------------------------
|ville     |nom        |character varying|None          |NO         |50                      |None             |
|ville     |population |numeric          |None          |YES        |None                    |10               |
|ville     |pays       |character varying|None          |YES        |50                      |None             |
|pays      |nom        |character varying|None          |NO         |50                      |None             |
|pays      |capitale   |character varying|None          |YES        |50                      |None             |


### MISE À JOUR DE TABLES

Considérer le schéma Entreprise du TD précédent rappelé ici :

- EMPLOYE (<u>NumSS</u>, NomE, PrenomE, NumChef*, VilleE, DateNaiss) 
- PROJET(<u>NumProj</u>, NomProj, RespProj*, VilleP, Budget) 
- EMBAUCHE (<u>NumSS*, NumProj*</u>, DateEmb, Profil*) 
- GRILLE_SAL (<u>Profil</u>, salaire)

La clé primaire de chaque relation est soulignée et les attributs des clés étrangères sont suivis d’un astérisque. Cette base contient des informations sur des employés et sur les projets dans lesquels ils sont impliqués. Ces employés sont embauchés dans un projet sur un profil donné et perçoivent un salaire en fonction de ce profil. Le chef d'un employé dans la table Employé et le chef d'un projet dans la table Projet sont des employés.

Le but de cet exercice est d'exprimer des instructions permettant d'insérer, de modifier et de supprimer des nuplets.

Dire à chaque fois si l'instruction exprimée est acceptée ou rejetée par le système en justifiant.

In [23]:
schemafile=open("TME9-creations-H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


In [24]:
show_schema(connection)

*********** Tables ************
4 rows
|table_name|
|-----------
|embauche  |
|employe   |
|projet    |
|grille_sal|


*********** Domaines ************
5 rows
|domain_name|check_clause                                 |
|----------------------------------------------------------
|dsal       |VALUE <= 90000                               |
|ddatenaiss |DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70    |
|dvilles    |LOWER(VALUE) IN('paris', 'lyon', 'marseille')|
|dnumproj   |CHAR_LENGTH(VALUE) >= 5                      |
|dnumss     |CHAR_LENGTH(VALUE) = 5                       |


*********** Attributs ************
17 rows
|table_name|column_name|data_type        |column_default|is_nullable|character_maximum_length|numeric_precision|
|---------------------------------------------------------------------------------------------------------------
|embauche  |numss      |numeric          |None          |NO         |None                    |5                |
|embauche  |numproj    |numeric     

#### INSERTIONS

##### Insérer l'employé identifié par '12456' qui se prénomme 'Alain'.


In [25]:
query="""
insert into employe(numss, prenomE) values(12456, 'Alain')
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc8246c70; closed: -1>

##### Insérer l'employée identifiée par '21456' qui s'appelle 'LARS Anna', qui habite 'Paris' et qui est née le 25-08-1975.

In [26]:
query="""
insert into employe(numss, nomE, prenomE, villeE, dateNaiss) values(21456, 'LARS', 'Anna', 'Paris', parsedatetime('25-08-1975', 'dd-MM-yyyy'))
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc04806d0; closed: -1>

##### Insérer le projet numéro '78143' dénommé 'ORCA' qui s'opère sous la responsabilité de 'Lars Anna' à Paris et qui a pour budget 250 000 euros.

In [27]:
query="""
insert into projet values(78143, 'ORCA', 21456, 'Paris', 250000)
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc04807c0; closed: -1>

##### Renseigner dans la base les salaires correspondants aux profils suivants : ('Responsable',80000), ('Développeur', 45000), ('Technicien',35000)

In [29]:
query="""
insert into grille_sal values ('Responsable',80000);
insert into grille_sal values ('Développeur', 45000); 
insert into grille_sal values ('Technicien',35000)
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc0480a90; closed: -1>

##### Renseigner dans la base le fait que 'Alain' a été embauché dans le projet 'ORCA' en tant que testeur en date du 01-04-2014.

In [35]:
query="""
insert into grille_sal values ('Testeur',null);
insert into embauche values (12456, 78143, PARSEDATETIME('01-04-2014', 'dd-MM-yyyy'),'Testeur')
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc0480d60; closed: -1>

##### Renseigner dans la base le fait que 'LARS Anna' fut embauchée dans le projet 'MEDUSA' en date du 28-02-2012 en tant que 'Développeur'.

In [37]:
query="""
insert into projet values(78144, 'MEDUSA', 21456, 'Paris', 250000);
insert into embauche values (21456, 78144, PARSEDATETIME('28-02-2012', 'dd-MM-yyyy'),'Développeur')
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc0495040; closed: -1>

#### SUPPRESSIONS

##### Supprimer les employés de plus de 67 ans

In [39]:
query="""
delete from employe where datediff(year,datenaiss, current_date)>67 
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc0495130; closed: -1>

##### Supprimer les employés prénommés 'Alain'

In [44]:
query="""
alter table embauche drop constraint fk_emb_emp;
alter table embauche add constraint fk_emb_emp foreign key(numss) REFERENCES employe(numss) on delete cascade on update cascade;
delete from employe where prenome = 'Alain'
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc0480c70; closed: -1>

##### Supprimer l'employée 'LARS Anna'

In [46]:
query="""
delete from employe where nome = 'LARS' and prenome = 'Alain'
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc0495220; closed: -1>

##### Supprimer les employés embauchés lors de la dernière année.

In [48]:
query="""
alter table projet drop constraint fk_resp;
alter table projet add constraint fk_resp foreign key(respproj) REFERENCES employe(numss) on delete set null on update cascade;
delete from employe where numss = any (select numss from embauche where datediff(year, dateemb,current_date)>0)
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc0495400; closed: -1>

##### Supprimer les projets dont la proportion des salaires dépasse la moitié du budget.

In [49]:
query="""
delete from projet p where (select sum(salaire) from embauche emb, grille_sal g where emb.numproj = p.numproj and g.profil = emb.profil)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc04957c0; closed: -1>

#### MISES À JOUR

##### Désormais on connait que l'employé Alain porte le nom BERNARD. Répercuter cette information.

In [51]:
query="""
select * from employe
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc04958b0; closed: -1>

##### L'employée LARS Anna est promue à la tête du projet identifié par 99245. Elle doit d'abord déménager avant de prendre ses responsabilités. Donner l'instruction qui modifie la ville de cette employée. On suppose qu'il existe un nuplet dans la table Projet avec les données suivantes : (99245,'MEDUSA',null,'Lyon',350000)

In [ ]:
query="""

"""
execute(connection,query,show=True)

##### Le budget de chaque projet équivaut au double des salaires de ses employés. Donner une instruction qui modifie les budgets des projets en conséquence.

In [ ]:
query="""

"""
execute(connection,query,show=True)

## Exercice TME: partie 1


In [52]:
schemafile=open("TME9-creations-H2.sql",mode="r",encoding='utf-8')
datafile=open("TME9-insertions-H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
insert_data=datafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
execute(connection, insert_data)
connection.commit()

0 rows
0 rows
0 rows


In [53]:
show_schema(connection)

*********** Tables ************
4 rows
|table_name|
|-----------
|embauche  |
|employe   |
|projet    |
|grille_sal|


*********** Domaines ************
5 rows
|domain_name|check_clause                                 |
|----------------------------------------------------------
|dsal       |VALUE <= 90000                               |
|ddatenaiss |DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70    |
|dvilles    |LOWER(VALUE) IN('paris', 'lyon', 'marseille')|
|dnumproj   |CHAR_LENGTH(VALUE) >= 5                      |
|dnumss     |CHAR_LENGTH(VALUE) = 5                       |


*********** Attributs ************
17 rows
|table_name|column_name|data_type        |column_default|is_nullable|character_maximum_length|numeric_precision|
|---------------------------------------------------------------------------------------------------------------
|embauche  |numss      |numeric          |None          |NO         |None                    |5                |
|embauche  |numproj    |numeric     

### Insertions rejetées (voir TME8)

#### Proposer une insertion dans la table Employé qui ne respecte pas la contrainte de clé primaire.

In [ ]:
query="""

"""
execute(connection,query,show=True)

#### Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de limite d'âge.

In [ ]:
query="""

"""
execute(connection,query,show=True)

#### Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de longueur de l'attribut NumSS.

In [ ]:
query="""

"""
execute(connection,query,show=True)

#### Proposer une insertion dans la table Employe qui ne respecte pas la contrainte sur les villes possibles.

In [ ]:
query="""

"""
execute(connection,query,show=True)

#### Insérer dans la table Employé deux employés avec le même nom et le même prénom.

In [ ]:
query="""

"""
execute(connection,query,show=True)

#### Proposer une insertion dans la table Grille_SAL qui ne respecte pas la contrainte sur le salaire qui ne doit pas dépasser 90 000.

In [ ]:
query="""

"""
execute(connection,query,show=True)

#### Proposer une insertion dans la table Projet qui ne respecte pas la contrainte référentielle vers Employe : insérer un responsable de projet qui n'est pas dans la table Employé

In [ ]:
query="""

"""
execute(connection,query,show=True)

#### Proposer une insertion dans la table Embauche qui ne respecte pas une des contraintes référentielles : par exemple, associer un employé existant à un projet qui n'existe pas.

In [ ]:
query="""

"""
execute(connection,query,show=True)

### Suppressions rejetées

In [62]:
schemafile=open("TME9-creations-H2.sql",mode="r",encoding='utf-8')
datafile=open("TME9-insertions-H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
insert_data=datafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
execute(connection, insert_data)
connection.commit()

0 rows
0 rows
0 rows


In [55]:
query="""
select * from Employe e
"""
execute(connection,query,show=True)

10 rows
|numss|nome     |prenome|numchef|villee   |datenaiss |
|-----------------------------------------------------
|22334|Adam     |Funk   |22334  |Paris    |1982-12-01|
|45566|Rachid   |Allaoui|22334  |Lyon     |1986-04-13|
|77889|Florent  |Girac  |45566  |Marseille|1990-11-04|
|90011|Mayla    |Aoun   |22334  |Lyon     |1987-03-26|
|22233|Christine|Lara   |45566  |Paris    |1982-08-09|
|34445|Amel     |Orlando|22334  |Lyon     |1976-02-14|
|55666|Mohsen   |Charef |45566  |Paris    |1991-05-28|
|77788|Tim      |Arabi  |22334  |Marseille|1984-06-08|
|89990|Fernando |Lopez  |45566  |Lyon     |1993-10-05|
|11122|Ada      |Tan Lee|22334  |Marseille|1994-03-21|


<cursor object at 0x7facc04959a0; closed: -1>

#### Proposer une suppression de nuplets de Employe qui respecte les contraintes d'intégrité.

In [56]:
query="""
delete from employe where numss=11122
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7facc0495310; closed: -1>

#### Proposer une suppression de nuplets de Employe qui ne respecte pas une contrainte référentielle d'une autre table. Précisez laquelle puis essayer de contourner les contraintes jusqu'à pouvoir supprimer l'Employe référencé.

In [57]:
query="""
delete from employe where numss=22334
"""
execute(connection,query,show=True)

The error 'Intégrité référentielle violation de contrainte: "fk_chef: public.employe FOREIGN KEY(numchef) REFERENCES public.employe(numss) (CAST(22334 AS NUMERIC(5)))"
Referential integrity constraint violation: "fk_chef: public.employe FOREIGN KEY(numchef) REFERENCES public.employe(numss) (CAST(22334 AS NUMERIC(5)))"; SQL statement:

delete from employe where numss=2233 [23503-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Intégrité référentielle violation de contrainte: "fk_chef: public.employe FOREIGN KEY(numchef) REFERENCES public.employe(numss) (CAST(22334 AS NUMERIC(5)))"
Referential integrity constraint violation: "fk_chef: public.employe FOREIGN KEY(numchef) REFERENCES public.employe(numss) (CAST(22334 AS NUMERIC(5)))"; SQL statement:

delete from employe where numss=2233 [23503-214]
' occurred


#### Même question pour Projet.

In [59]:
query="""
delete from Projet where numProj = 12333;
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc0495a90; closed: -1>

#### Même question pour Grille_sal.

In [61]:
query="""
delete from Grille_Sal where profil = 'Admin';
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc04955e0; closed: -1>

#### Est-ce possible de supprimer les nuplets de Embauche sans rencontrer les mêmes problèmes que précédemment ?

In [ ]:
query="""

"""
execute(connection,query,show=True)

### Mises à jour rejetées

#### Proposez un mise à jour de Employe qui ne respecte pas les contraintes référentielles puis tenter de trouver une mise-a-jour qui les respecte toutes.

In [63]:
query="""
update Employe set numSS = 12345 where numSS = 22334
"""
execute(connection,query,show=True)

The error 'Intégrité référentielle violation de contrainte: "fk_chef: public.employe FOREIGN KEY(numchef) REFERENCES public.employe(numss) (CAST(22334 AS NUMERIC(5)))"
Referential integrity constraint violation: "fk_chef: public.employe FOREIGN KEY(numchef) REFERENCES public.employe(numss) (CAST(22334 AS NUMERIC(5)))"; SQL statement:

update Employe set numSS = 12345 where numSS = 2233 [23503-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Intégrité référentielle violation de contrainte: "fk_chef: public.employe FOREIGN KEY(numchef) REFERENCES public.employe(numss) (CAST(22334 AS NUMERIC(5)))"
Referential integrity constraint violation: "fk_chef: public.employe FOREIGN KEY(numchef) REFERENCES public.employe(numss) (CAST(22334 AS NUMERIC(5)))"; SQL statement:

update Employe set numSS = 12345 where numSS = 2233 [23503-214]
' occurred


In [64]:
query="""
update Employe set numSS = 34445 where numSS = 97332
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc0495c70; closed: -1>

#### Même question avec Projet.

In [65]:
query="""
update Projet set numProj = 80066 where numProj = 12333
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc0495d60; closed: -1>

#### Proposez une mise-a-jour de Projet qui ne respecte pas la contrainte resprojet selon laquelle le responsable d'un projet habite la ville du projet dont il est responsable.

In [66]:
query="""
update Projet set villeP = 'Marseille' where numProj = 12333
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7facc0495e50; closed: -1>

#### Proposez une mise-a-jour de profil dans Grille_sal qui ne respecte pas les contraintes référentielles.

In [67]:
query="""
update Grille_Sal set profil = 'Dev' where profil = 'Deve'
"""
execute(connection,query,show=True)

The error 'Intégrité référentielle violation de contrainte: "fk_emb_sal: public.embauche FOREIGN KEY(profil) REFERENCES public.grille_sal(profil) ('Deve')"
Referential integrity constraint violation: "fk_emb_sal: public.embauche FOREIGN KEY(profil) REFERENCES public.grille_sal(profil) ('Deve')"; SQL statement:

update Grille_Sal set profil = 'Dev' where profil = 'Deve [23503-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Intégrité référentielle violation de contrainte: "fk_emb_sal: public.embauche FOREIGN KEY(profil) REFERENCES public.grille_sal(profil) ('Deve')"
Referential integrity constraint violation: "fk_emb_sal: public.embauche FOREIGN KEY(profil) REFERENCES public.grille_sal(profil) ('Deve')"; SQL statement:

update Grille_Sal set profil = 'Dev' where profil = 'Deve [23503-214]
' occurred


### Exercie TME: Partie 2

Le but de cette partie d'illustrer un cas où les mises-a-jour ne sont pas rejetées mais plutôt propagées.


In [68]:
schemafile=open("TME9-creations-cascade-H2.sql",mode="r",encoding='utf-8')
datafile=open("TME9-insertions-H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
insert_data=datafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
execute(connection, insert_data)
connection.commit()

0 rows
0 rows
0 rows


In [69]:
query="""
select * from Employe
"""
execute(connection,query,show=True)

10 rows
|numss|nome     |prenome|numchef|villee   |datenaiss |
|-----------------------------------------------------
|22334|Adam     |Funk   |22334  |Paris    |1982-12-01|
|45566|Rachid   |Allaoui|22334  |Lyon     |1986-04-13|
|77889|Florent  |Girac  |45566  |Marseille|1990-11-04|
|90011|Mayla    |Aoun   |22334  |Lyon     |1987-03-26|
|22233|Christine|Lara   |45566  |Paris    |1982-08-09|
|34445|Amel     |Orlando|22334  |Lyon     |1976-02-14|
|55666|Mohsen   |Charef |45566  |Paris    |1991-05-28|
|77788|Tim      |Arabi  |22334  |Marseille|1984-06-08|
|89990|Fernando |Lopez  |45566  |Lyon     |1993-10-05|
|11122|Ada      |Tan Lee|22334  |Marseille|1994-03-21|


<cursor object at 0x7facc04a3130; closed: -1>

### Mises-à-jour

#### Reéxécutez les mise-à-jour et les suppressions de la Partie 1 et verifiez ce qui se passe.

#### Mettez à jour le numéro d'un employé reférencé par d'autres nuplets

In [ ]:
query="""
update Employe set ....
"""
execute(connection,query,show=True)

In [ ]:
query="""
select * from Employe
"""
execute(connection,query,show=True)

### Suppression

#### Supprimer les nuplets de Employe et vérifier que tous les nuplets des autres tables qui référencent des employes venant d'être supprimés seront supprimés eux aussi.

In [ ]:
query="""
delete from Employe ....
"""
execute(connection,query,show=True)

In [ ]:
query="""
select * from Employe
"""
execute(connection,query,show=True)

# Fermer la connexion

In [70]:
connection.commit() # implicit avec close
connection.close()

In [71]:
connection

<connection object at 0x7facc9f3ba40; dsn: 'user=ba password=xxx dbname=jo5613 host=localhost port=5613', closed: 1>